# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902012


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:05<01:15,  2.68s/it]

Rendering models:  10%|█         | 3/30 [00:05<00:52,  1.94s/it]

Rendering models:  13%|█▎        | 4/30 [00:05<00:36,  1.40s/it]

Rendering models:  20%|██        | 6/30 [00:05<00:24,  1.01s/it]

...

Rendering models:  90%|█████████ | 27/30 [00:11<00:00,  5.55it/s]

Rendering models:  93%|█████████▎| 28/30 [00:11<00:00,  5.15it/s]

Rendering models:  97%|█████████▋| 29/30 [00:12<00:00,  5.48it/s]

Rendering models: 100%|██████████| 30/30 [00:12<00:00,  4.05it/s]

taisribeiro                           0.002514
klmasters                             0.000366
WhittleMario                          0.000360
esazure                               0.000107
start056                              0.000134
smilingsloth.                         0.000557
aimee12                               0.000365
bronzesword                           0.000291
not-logged-in-545b59d533d78a4f8dea    0.000336
not-logged-in-7cc17a8a1e6ff5a23d80    0.000297
not-logged-in-7513f0d73e7891ae677f    0.007877
Jessica_Korkmaz                       0.004308
not-logged-in-314d48142a3a122ba666    0.001409
crush202020                           0.000151
AlexYoung35                           0.000204
BraxtonKinney                         0.000242
pangeli5                              0.000088
not-logged-in-2c4a9bf7eca3b2d9e58f    2.682611
not-logged-in-3569dcff4dbd0ee90f50    0.001914
Lavadude                              0.058129
helios1235                            0.000142
TateNachampas

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())